In [58]:
# imports

import os
import json
from dotenv import load_dotenv
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
from agents.deals import Deal, ScrapedDeal, DealSelection

In [60]:
import importlib
import agents.deals
importlib.reload(agents.deals)
from agents.deals import Deal,ScrapedDeal, DealSelection

In [ ]:
import importlib
import agents.scanner_agent
importlib.reload(agents.deals)
from agents.deals import Deal,ScrapedDeal, DealSelection

In [26]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gemini-2.5-flash'
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

In [27]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:59<00:00, 23.94s/it]


In [28]:
len(deals)

50

In [29]:
deals[1].describe()

'Title: Walmart Deals Week TV Sale: Up to 72% off + free shipping w/ $35\nDetails: There are some really strong deals to be had here, with prices on smaller models starting well below $100. We\'ve Samsung Q60DB\xa0Series QN75Q60DBFXZA\xa075" Class 4K HDR 10+ QLED Smart TV for $797.99 ($400 off.) Shop Now at Walmart\nFeatures: \nURL: https://www.dealnews.com/Walmart-Deals-Week-TV-Sale-Up-to-72-off-free-shipping-w-35/21751254.html?iref=rss-c142'

In [30]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [31]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [32]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: PWRcard 2,500mAh Slim Portable Power Bank 4-Pack for $12 + free shipping
Details: As one of the best deals today at MorningSave, get this 4-pack at 90% off. Plus, use coupon code "DEALNEWS" to get free shipping, an additional $8.99 savings. Buy Now at MorningSave
Features: 
URL: https://www.dealnews.com/PWRcard-2-500-m-Ah-Slim-Portable-Power-Bank-4-Pack-for-12-free-

In [33]:
def get_recommendations():
    try:
        model = genai.GenerativeModel(MODEL)
        response = model.generate_content(user_prompt)
        result = response.text
    except Exception as e :
        self.log(f"Gemini API Error: {e}")
        
    return result

In [65]:
def parse_price(price_str: str) -> float:
    """
    Parse price string like "$210" to float
    """
    if isinstance(price_str, str):
        # Remove $ and any commas, convert to float
        return float(price_str.replace('$', '').replace(',', ''))
    return float(price_str)

def extract_json(text: str) -> str:
    """
    Extract JSON from text that might contain markdown code blocks
    """
    # Remove markdown code blocks
    text = re.sub(r'```json\n', '', text)
    text = re.sub(r'```\n?', '', text)
    text = text.strip()
    
    # Try to find JSON content - check for both objects {} and arrays []
    object_start = text.find('{')
    array_start = text.find('[')
    
    # Determine which comes first (or if only one exists)
    if object_start != -1 and (array_start == -1 or object_start < array_start):
        # JSON object
        start = object_start
        end = text.rfind('}') + 1
    elif array_start != -1:
        # JSON array
        start = array_start
        end = text.rfind(']') + 1
    else:
        # No JSON found, return as is
        return text
    
    if start != -1 and end != 0:
        return text[start:end]
    return text

def get_recommendations():
    try:
        # This would be your actual Gemini API call
        model = genai.GenerativeModel(MODEL)
        response = model.generate_content(user_prompt)
        reply = response.text
        
        
        print("RAW Gemini reply:\n", repr(reply))  # Debug: show raw text
        clean_text = extract_json(reply)
        print("Cleaned JSON text:\n", clean_text)  # Debug: show stripped version
        parsed = json.loads(clean_text)
        print("Parsed JSON type:", type(parsed))  # Debug: show type
        
        # Determine deal list location
        deals_data = None
        if isinstance(parsed, dict):
            print("Parsed keys:", parsed.keys())  # Extra debug
            for key in ["selected_deals", "deals", "promising_deals"]:
                deals_data = parsed.get(key)
                if deals_data:
                    break
        elif isinstance(parsed, list):
            deals_data = parsed
        else:
            raise ValueError("Parsed JSON is not a list or dict")
        
        if not deals_data:
            raise ValueError("No deals found in parsed JSON")
        
        deals = [
            Deal(
                title=deal["title"],
                product_description=deal["product_description"],
                price=parse_price(deal["price"]),
                url=deal.get("url")  # Use .get() since URL might not exist
            )
            for deal in deals_data
        ]
        
        return DealSelection(deals=deals)
        
    except json.JSONDecodeError as e:
        print(f"❌ JSON parsing error: {e}")
    except Exception as e:
        print(f"❌ Error: {e}")
    return None

In [66]:
result = get_recommendations()

RAW Gemini reply:
 '```json\n[\n  {\n    "title": "Refurb Unlocked Samsung Galaxy S21 256GB 5G Android Smartphone",\n    "price": "$210",\n    "product_description": "This is a refurbished Samsung Galaxy S21, an unlocked 5G Android smartphone featuring Android 11. It\'s equipped with a powerful Qualcomm SM8350 Snapdragon 888 Octa-Core Processor, enabling 8K video recording. The device boasts a vibrant 6.2-inch 2400x1080 120Hz touchscreen display. For photography, it includes a versatile camera system with a 64MP telephoto lens, 12MP wide and ultra-wide cameras, and a 10MP front camera. The specific model is SM-G991UZAEXAA."\n  },\n  {\n    "title": "HP Victus Ryzen 7 15.6\\" Laptop w/ NVIDIA GeForce RTX 4050",\n    "price": "$650",\n    "product_description": "The HP Victus is a high-performance 15.6-inch laptop designed for gaming and demanding tasks. It is powered by an AMD Ryzen 7 7445HS 3.1GHz 8-core CPU, complemented by 16GB of RAM and a 512GB SSD for fast storage and multitasking

In [67]:
print(result)

deals=[Deal(product_description="This is a refurbished Samsung Galaxy S21, an unlocked 5G Android smartphone featuring Android 11. It's equipped with a powerful Qualcomm SM8350 Snapdragon 888 Octa-Core Processor, enabling 8K video recording. The device boasts a vibrant 6.2-inch 2400x1080 120Hz touchscreen display. For photography, it includes a versatile camera system with a 64MP telephoto lens, 12MP wide and ultra-wide cameras, and a 10MP front camera. The specific model is SM-G991UZAEXAA.", price=210.0, url=None), Deal(product_description='The HP Victus is a high-performance 15.6-inch laptop designed for gaming and demanding tasks. It is powered by an AMD Ryzen 7 7445HS 3.1GHz 8-core CPU, complemented by 16GB of RAM and a 512GB SSD for fast storage and multitasking. The visual experience is enhanced by a 15.6-inch 1920x1080 (1080p) IPS display, and for graphics-intensive applications, it features a dedicated NVIDIA GeForce RTX 4050 6GB graphics card. The laptop runs on Windows 11 Hom

In [68]:
len(result.deals)

5

In [75]:
result.deals[3]

Deal(product_description='The Birdfy Smart Bird Feeder offers a unique way to observe local wildlife with its integrated camera and advanced AI species identification. It requires a 2.4GHz Wi-Fi router for connectivity, allowing users to stream and record full HD 1080p video with a 135° wide-angle lens. The feeder also features night vision, ensuring clear footage day or night, and its AI can identify thousands of different bird species. This smart feeder model is known as Birdfy Netvue2.', price=106.0, url=None)

In [79]:
from agents.scanner_agent import ScannerAgent

In [80]:
agent = ScannerAgent()
result = agent.scan_gemini()

NameError: name 'collection' is not defined